# 2. MalConv를 통한 악성여부 확인

* MalConv Class를 만들기 위한 package 로드

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. MalConv 만들기

* MalConv Class 만들기

In [2]:
class MalConv(nn.Module):
    def __init__(self, out_size=2, channels=128, window_size=512, embd_size=8):
        
        ### MalConv 제작을 위한 variable 초기화 ###

        # nn.Module 상속 선언
        super(MalConv, self).__init__()
        
        # Embedding Vector 구성(257(0~256) * embd_size)
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        
        # window_size 선언     
        self.window_size = window_size
        
        # 2개 1D Convolution Layer 선언(embd_size * channels * window_size)
        self.conv_1 = nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)        
        
        # MaxPooling Layer 선언
        self.pooling = nn.AdaptiveMaxPool1d(1)        

        # Fully Connected Layer 선언(channels * channels)
        self.fc_1 = nn.Linear(channels, channels)
        
        # Fully Connected Layer 선언(channels * out_size)
        self.fc_2 = nn.Linear(channels, out_size)
    
    ### Malconv 실행 시 동작 함수 ###
    def forward(self, x):
        
        # TODO
        # input x에 대한 embedding vector 구축
        x = self.embd(x.long())
        
        # CNN 연산을 위한 차원 교환
        x = torch.transpose(x,-1,-2)

        # conv_1을 이용한 cnn값 추출
        cnn_value = ### FILL HERE ###

        # conv_2 및 sigmoid를 이용한 값 추출
        gating_weight = torch.sigmoid(self.conv_2(x))

        # cnn_value와 gating_weight 추출된 값의 합성곱 연산 수행
        x = ### FILL HERE ###      
        
        # 합성곱 값에 대한 MaxPooling 수행
        x = self.pooling(x)

        # FC layer 연산을 위한 shape 변경
        x = x.view(x.size(0), -1)

        # fc_1을 통한 FC layer 연산 및 Relu 활성화 함수 연산
        x = F.relu(### FILL HERE ###)

        # ReLU 함수값을 fc_2를 통한 FC layer 연산 수행
        x = ### FILL HERE ###  
        return x

## 2. MalConv 모델 이용하기

### 1. MalConv 모델 이용 준비
* 앞서 구축한 MalConv class 불러오기

In [3]:
# TODO: 256 channel, 512 window_size, embed_size 8의 Malconv 모델 구성
malconv_model = MalConv(### FILL HERE ###)

* 기 구축한 문서형 Stream data의 MalConv 모델 가중치 불러오기 및 MalConv 적용

In [ ]:
# TODO: weight load 및 모델 적용
weight = torch.load("../data/malconv_doc.pth", map_location=torch.device('cpu'))
### FILL HERE ###

### 2. MalConv를 이용한 문서형 stream data의 악성 여부 확인

* 정상/악성 Stream data 불러오기

In [5]:
with open("../data/2_benign.txt", "r") as f:
    benign_data = f.read().split(",")
    benign_data = [int(x) for x in benign_data]

In [6]:
with open("../data/2_critical.txt", "r") as f:
    mal_data = f.read().split(",")
    mal_data = [int(x) for x in mal_data]

* 악성 탐지를 위한 과정 구축

In [7]:
def detect(malconv, stream_data):
    """
    :param nn.Module malconv: 앞서 구축한 MalConv 모델
    :param list stream_data : 악성 여부를 확인할 stream data
    """
    # TODO
    ### MalConv를 이용한 Stream data 악성 여부 확인 함수 만들기###    
    # stream data를 malconv에 맞는 형식으로 변환
    stream_data = torch.from_numpy(np.frombuffer(bytearray(stream_data), dtype=np.uint8)[np.newaxis, :])

    # malconv를 통해 값 도출
    output = ### FILL HERE ###

    # 도출된 값을 Softmax 함수를 이용한 확률값으로 변환
    output = F.softmax(output, dim=-1).detach().numpy()[0,1]
    return output

* 정상/악성 Stream Data 악성 탐지 여부 확인

In [ ]:
detect(malconv_model, benign_data)

In [ ]:
detect(malconv_model, mal_data)